In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
import numpy as np
from scipy.interpolate import RectBivariateSpline
from matplotlib import cm
from matplotlib.ticker import MaxNLocator
import sys, os, io, string, shutil, math
import glob
import re

## Growth Kinetics

In [ ]:
#growth kinetics
df = pd.read_csv(
    "cluster.dat",
    sep=r"\s+",
    comment="#",
    header=None,
    names=["label", "time_ns", "cluster_avg", "std"],
    engine="python",
)

# Clean & types
df = df[pd.to_numeric(df["time_ns"], errors="coerce").notna()].copy()
df["time_ns"] = df["time_ns"].astype(float)
df["cluster_avg"] = pd.to_numeric(df["cluster_avg"], errors="coerce")
df["std"] = pd.to_numeric(df["std"], errors="coerce")
df["std"] = df["std"].fillna(0.0)

# Bounds
df["lower"] = df["cluster_avg"] - df["std"]
df["upper"] = df["cluster_avg"] + df["std"]

# --- Plot ---
plt.figure(figsize=(12, 8))
LINE_WIDTH = 2.5
ERROR_ALPHA = 0.2

for label, g in df.groupby("label"):
    g = g.sort_values("time_ns")
    line, = plt.plot(
        g["time_ns"].to_numpy(),
        g["cluster_avg"].to_numpy(),
        label=str(label),
        linewidth=LINE_WIDTH,
        zorder=3
    )
    plt.fill_between(
        g["time_ns"].to_numpy(),
        g["lower"].to_numpy(),
        g["upper"].to_numpy(),
        color=line.get_color(),
        alpha=ERROR_ALPHA,
        zorder=2
    )

plt.xlabel("Simulation Time (ns)", fontsize=16)
plt.ylabel("Completion Fraction", fontsize=16)
plt.grid(True, alpha=0.4)
plt.legend(fontsize=12, framealpha=0.9, loc="upper left")
plt.tight_layout()
#plt.savefig("growth_kinetics_from_cluster.png", dpi=300, bbox_inches="tight")
plt.show()

## Mass Spectrum

In [ ]:
def alphanum_key(s):
    """Natural sorting key function"""
    return [int(c) if c.isdigit() else c.lower() for c in re.split('([0-9]+)', s)]

def Merge(dict1, dict2):
    for k in dict2.keys():
        if k in dict1.keys():
            dict1[k] += dict2[k] 
        else:
            dict1[k] = dict2[k]
    return dict1

def padding(dict1, rangelen):
    len_value = np.linspace(1, rangelen, rangelen)
    for i in len_value:
        if i not in dict1.keys():
            dict1[i] = 0
    return dict1

def compute_cluster_size(file_path, num_of_units, max_cluster_size=20):
    # Find all cluster files sorted naturally
    p_data_list = sorted(glob.glob(os.path.join(file_path, "cluster*")), 
                         key=alphanum_key)
    
    # Initialize data containers
    cluster_avg = []
    max_cluster = []
    weighted_avg = []
    cluster_distr = {}
    
    # Initialize 3D mass spectrum matrix: [time, size, mass]
    time_points = len(p_data_list)
    mass_matrix = np.zeros((time_points, max_cluster_size))
    
    # Process each time point
    for i, file_name in enumerate(p_data_list):
        # Read cluster data
        cluster_data = pd.read_csv(file_name, sep="\s+", comment='#', 
                                 skiprows=2, header=None, 
                                 names=["id", "value"])
        
        # Convert values to cluster sizes (in units)
        sizes = cluster_data["value"].to_numpy() / num_of_units
        
        # Calculate metrics
        cluster_avg.append(np.mean(sizes))
        max_cluster.append(np.max(sizes))
        weighted_avg.append(np.sum(cluster_data["value"]) / num_of_units / len(sizes))
        
        # Calculate mass distribution for current time point
        size_counts = {}
        for size in sizes:
            # Only consider clusters within our size range
            int_size = int(round(size))
            if 1 <= int_size <= max_cluster_size:
                size_counts[int_size] = size_counts.get(int_size, 0) + size * num_of_units
        
        # Pad missing sizes with zeros
        size_counts = padding(size_counts, max_cluster_size)
        
        # Add to mass matrix
        for size in range(1, max_cluster_size + 1):
            mass_matrix[i, size-1] = size_counts[size]
        
        # Update distribution for last 50 steps
        if i >= (len(p_data_list) - 50):
            # Create normalized count distribution
            count_distr = {int(round(size)): count for size, count in zip(*np.unique(sizes, return_counts=True))}
            Merge(cluster_distr, count_distr)
    
    # Average the distribution over last 50 time steps
    for k in cluster_distr:
        cluster_distr[k] /= 50
    
    return cluster_avg, max_cluster, weighted_avg, cluster_distr, mass_matrix

In [ ]:
def plot_smooth_mass_spectrum(mass_matrix, max_size):
    time_points = np.arange(mass_matrix.shape[0])
    cluster_sizes = np.arange(1, max_size + 1)
    
    fine_time = np.linspace(0, time_points.max(), 300)
    fine_size = np.linspace(1, max_size, 300)
    
    interp_func = RectBivariateSpline(time_points, cluster_sizes, mass_matrix, kx=3, ky=3)
    Z_fine = interp_func(fine_time, fine_size)
    X_fine, Y_fine = np.meshgrid(fine_time, fine_size, indexing='ij')
    
    # Create plot
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d', facecolor='none')
    
    # Remove background and grid
    ax.grid(False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    # Set specific tick positions for cluster sizes
    y_ticks = np.arange(1, max_size + 1, max(1, max_size//10))
    ax.set_yticks(y_ticks)
    
    surf = ax.plot_surface(
        X_fine, Y_fine, Z_fine,
        cmap='viridis',
        edgecolor='none',  # Remove edge lines for smoother appearance
        alpha=0.92,
        rstride=3,        # Reduce sampling for better performance
        cstride=3,
        antialiased=True   # Enable anti-aliasing
    )
    
    #scale_factor = 1200/(0.003 * 2.75 * 10e-3 * 1.2e8)
    scale_factor = 1200/1000
    ax.xaxis.set_major_locator(MaxNLocator(nbins=5))
    xticks = ax.get_xticks()
    ax.set_xticklabels((xticks / scale_factor).astype(int))  # rescale tick label
    ax.set_xlabel('Simulation Time (ns)', labelpad=18, fontsize=14)
    ax.set_ylabel('Cluster Size', labelpad=18, fontsize=14)
    ax.set_zlabel('Mass Fraction', labelpad=18, fontsize=14)
    
    # Add color bar
    #cbar = fig.colorbar(surf, ax=ax, pad=0.15, shrink=0.7)
    #cbar.set_label('Mass Density', fontsize=12, fontweight='bold')
    
    # Adjust viewing angle for best perspective
    ax.view_init(elev=32, azim=-52)
    
    # Improve tick visibility
    ax.tick_params(axis='both', which='major', labelsize=10, pad=8)
    
    plt.tight_layout()

    #plt.savefig('mass_spectrum_b500.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Calculate metrics and get mass matrix
file_path = "bd_b5_e2.3/"
num_of_units = 105
max_size = 20

cluster_avg, max_cluster, weighted_avg, cluster_distr, mass_matrix = compute_cluster_size(
    file_path, num_of_units, max_size
)

In [ ]:
plot_smooth_mass_spectrum(mass_matrix/np.max(mass_matrix), max_size=20)